In [1]:
from __future__ import division
from __future__ import print_function

import sys
import os
import argparse
import time
import random
from glob import glob
from datetime import datetime

import align.detect_face
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import misc
from scipy.interpolate import Rbf
from skimage import io, transform
from six.moves import xrange

import tensorflow as tf
import tensorflow.contrib.slim as slim

import facenet
import helper

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

plt.ion() # interactive mode

In [3]:
landmarks_frame = pd.read_csv('./face_landmarks_generate.csv')
file_list = landmarks_frame.image_name.values.tolist()

In [4]:
print(len(file_list))

7867


In [29]:
dataset = helper.Dataset('nf',file_list, 160)

In [31]:
batch_size = 100

In [30]:
f_emb_g = tf.Graph()
f_emb_list = []
f_avgP_list = []

In [33]:
with f_emb_g.as_default():    
    with tf.Session() as sess:
        start_load_facenet = time.time()
        print("--- %s start load facenet ---" % (start_load_facenet))
        facenet.load_model('./20171012')
        print("--- %s facenet loaded ---" % (time.time() - start_load_facenet))

        # Get input and output tensors
        f_inputs = tf.get_default_graph().get_tensor_by_name("input:0")
        f_phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        f_avgP_logits = tf.get_default_graph().get_tensor_by_name("InceptionResnetV1/Logits/AvgPool_1a_8x8/AvgPool:0")
        f_emb_logits = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        
        start_test = time.time()
        
        total_size = int(len(file_list)/batch_size)
        for i, batch in enumerate(dataset.get_batches(batch_size)):
            f_feed = {f_inputs: batch, f_phase_train_placeholder:False}
            f_embedings, f_avgPool = sess.run([f_emb_logits, f_avgP_logits], feed_dict=f_feed)
            f_emb_list.append(f_embedings)
            f_avgP_list.append(f_avgPool)
            sys.stdout.write('\r'+ "[%d/%d] %s" % (i, total_size, time.time() - start_test))
            print()
        print("--- %s seconds ---" % (time.time() - start_test))

--- 1508420512.736853 start load facenet ---
Model filename: ./20171012
--- 11.739103078842163 facenet loaded ---
[0/78] 25.909481287002563
[1/78] 43.19308114051819
[2/78] 59.66143012046814


KeyboardInterrupt: 

In [ ]:
np.savez('f_emb_list.npz', *f_emb_list)
np.savez('f_avgP_list.npz', *f_avgP_list)